In [ ]:
! pip install -U accelerate
! pip install -U transformers
! pip install datasets


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EvalPrediction
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import pipeline
import time
from scipy.special import softmax

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load dataset

In [20]:
import pandas as pd

df_path = "SET_PATH_HERE"
df = pd.read_csv(df_path)
df.head()



,Unnamed: 0,text,sentiment
0,0,"My daughter liked it but I was aghast, that a ...",0
1,1,I... No words. No words can describe this. I w...,0
2,2,this film is basically a poor take on the old ...,0
3,3,"This is a terrible movie, and I'm not even sur...",0
4,4,First of all this movie is a piece of reality ...,1


In [21]:
df['text'] = df.text.str.lower()

In [22]:
df = df[:12000]

In [ ]:
df

,Unnamed: 0,text,sentiment
0,0,"my daughter liked it but i was aghast, that a ...",0
1,1,i... no words. no words can describe this. i w...,0
2,2,this film is basically a poor take on the old ...,0
3,3,"this is a terrible movie, and i'm not even sur...",0
4,4,first of all this movie is a piece of reality ...,1
...,...,...,...
11995,11995,i don't know why this has gotten any decent re...,0
11996,11996,"surprisingly good ""mean streets""-type crime dr...",1
11997,11997,this film basically try to portray the heroism...,0
11998,11998,a few buddies and myself have the strange hobb...,0


In [23]:
from datasets import load_dataset, Dataset

df = Dataset.from_pandas(df)

## Run training

In [8]:
df = df.train_test_split(test_size=0.3)

In [9]:
model_name = 'sentence-transformers/all-mpnet-base-v2'

tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments


model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-mpnet-base-v2 and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding=True, truncation=True)

train_tokenized = df['train'].map(tokenize_function, batched=True)
test_tokenized = df['test'].map(tokenize_function, batched=True)

# Prepare the datasets
def format_dataset(dataset):
    return {'input_ids': dataset['input_ids'], 'attention_mask': dataset['attention_mask'], 'labels': dataset['sentiment']}

train_tokenized = train_tokenized.map(format_dataset)
test_tokenized = test_tokenized.map(format_dataset)

In [ ]:
import evaluate
import numpy as np
metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
args = TrainingArguments(
      output_dir=f"./output",
      overwrite_output_dir = 'True',
      evaluation_strategy="epoch",
      eval_steps=40,
      logging_steps = 40,
      learning_rate = 5e-5,
      per_device_train_batch_size=16,
      per_device_eval_batch_size=16,
      num_train_epochs=3,
      seed=0,
      save_total_limit = 2,
      metric_for_best_model = 'f1',
      load_best_model_at_end=True,
      save_strategy = "epoch"
  )
trainer = Trainer(
      model=model,
      args=args,
      train_dataset=train_tokenized,
      eval_dataset=test_tokenized,
      #compute_metrics=compute_metrics,

  )
trainer.train()


In [ ]:
trainer.save_model(f'./output_3')

In [ ]:
best_ckpt_path = trainer.state.best_model_checkpoint

## Run inference

In [ ]:
model_name = 'sentence-transformers/all-mpnet-base-v2'

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [13]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding=True, truncation=True)


In [14]:
def format_dataset(dataset):
    return {'input_ids': dataset['input_ids'], 'attention_mask': dataset['attention_mask']}

In [10]:
model_pred = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Colab/output/checkpoint-1800', local_files_only=True)

#Normal predictions

In [29]:

def predict_df(df):
    start = time.time()
    train_tokenized = df.map(tokenize_function, batched=True)
    train_tokenized = train_tokenized.map(format_dataset)
    trainer = Trainer(model=model_pred, tokenizer=tokenizer)
    predictions = trainer.predict(test_dataset=train_tokenized).predictions
    # Adding a softmax layer to get probabilities. If you want class labels instead -  np.argmax(predictions, axis=1)
    predictions = np.array([softmax(element) for element in predictions])[:, 1]
    end = time.time()

    print('time taken', str(end-start))
    return predictions


predict_df(df)

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

time taken 468.13907527923584


array([0.00055363, 0.00055024, 0.00055061, ..., 0.00055136, 0.00055194,
       0.00055187], dtype=float32)

##Faster predictions

In [26]:
import multiprocessing
from functools import partial
import numpy as np

def parallel_tokenize_function(df_row, tokenize_func):
    return tokenize_func(df_row)

def predict_df(df, batch_size=32, num_processes=None):
    start = time.time()
    if num_processes is None:
        num_processes = multiprocessing.cpu_count()

    with multiprocessing.Pool(processes=num_processes) as pool:
        partial_tokenize = partial(parallel_tokenize_function, tokenize_func=tokenize_function)
        train_tokenized = pool.map(partial_tokenize, df, chunksize=batch_size)

        train_tokenized = [format_dataset(row) for row in train_tokenized]

        trainer = Trainer(model=model_pred, tokenizer=tokenizer)
        predictions = trainer.predict(test_dataset=train_tokenized).predictions
        predictions = np.array([softmax(element) for element in predictions])[:, 1]
    end = time.time()

    print('time taken', str(end-start))
    return predictions

predict_df(df)

time taken 423.79246950149536


array([0.00055363, 0.00055024, 0.00055061, ..., 0.00055136, 0.00055194,
       0.00055187], dtype=float32)

## Depending on the number of allocated threads, the speedup is around 20s to 1min